### Jolly Seber 

&nbsp;

Jolly-Seber model a statistical method to estimate the size of animal populations based upon mark-recapture data. The process is straight forward. In the first step, a sample of the population is captured and marked with chips or tags. After marking, the captured individuals are released back into the wild. After an interval of time, another sample is taken which may include both previously captured (marked) and new (unmarked) individuals. The model assumes equal capture probability between marked and unmarked so the ratio between marked and unmarked can derive the potential population size. However, the model assumes closed population without migration, birth or death. Many advanced models take into account of individual heterogeneity in capture and survival probabilities and time-varying capture and survival rates. These models can provide more accurate and precise population estimates while addressing the limitations of the basic Jolly-Seber model.

In [1]:
import os
import pandas as pd
import numpy as np
import statsmodels.api as sm
from pyproj import Proj
import math
import pyodbc
os.chdir('C:/Users/tm/Downloads/utas/WildlifeDatabases')

### cleanse

In [2]:
#  './arthur river/ARV_database_2022_11_CS.accdb','./black river/BRI_database_2022_11_CS.accdb',
#          './freycinet/FNP_database_2022_11_KJS.accdb','./takone/TKN_database_2022_11_KJS.accdb',
#          './west pencil pine/WPP_September_2022_11_KJS.accdb','./wukalina/Wukalina_NOV2020_SN.accdb'
#  './crabtree/CBT_database_2022_11_KJS.accdb','./franklin/FRA_database_devil_2022_11_KJS.accdb',

In [3]:
traphist=pd.DataFrame(columns=['ID', 'TrapID', 'DateOfUse', 'Event', 'CaptureType', 'Microchip',
       'Comment'])

traps=pd.DataFrame(columns=['TrapID', 'Northing', 'Easting', 'Comments', 'TrapLine', 'Researcher'])

for i in [
         './woodbridge sandfly/Channel_database_devil_2022_06_KJS.accdb',
        ]:

    conn = pyodbc.connect(r'Driver={Microsoft Access Driver (*.mdb, *.accdb)};DBQ='+f'{i};')
    traps=pd.concat([traps,pd.read_sql('select * from traps',conn)])
    traphist=pd.concat([traphist,pd.read_sql('select * from traphistory',conn)])
    traps.reset_index(inplace=True,drop=True)
    traphist.reset_index(inplace=True,drop=True)

C:\Users\tm\AppData\Local\Temp\ipykernel_19604\1923287904.py:11: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  traps=pd.concat([traps,pd.read_sql('select * from traps',conn)])
C:\Users\tm\AppData\Local\Temp\ipykernel_19604\1923287904.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  traphist=pd.concat([traphist,pd.read_sql('select * from traphistory',conn)])


In [4]:
#focus on recapture
traphist=traphist[traphist['Event'].isin(['Tasmanian devil','Tasmanian Devil',])].copy()

#remove null microchip
traphist=traphist.loc[traphist['Microchip'].dropna().index]

#remove duplicates
traphist=traphist.loc[traphist[['Microchip','DateOfUse']].drop_duplicates().index]

In [5]:
#sort by date
grande=traphist[['Microchip','DateOfUse',]].sort_values(
    ['Microchip','DateOfUse'])
grande.reset_index(inplace=True,drop=True)

#datetimeindex
grande['DateOfUse']=pd.to_datetime(grande['DateOfUse'])

In [6]:
#for each year,each devil only counts once
grande['year']=grande['DateOfUse'].dt.year
pop_raw=grande.loc[grande[['year','Microchip']].drop_duplicates().index]

In [7]:
#sort by year
pop_raw=pop_raw.sort_values(['year','Microchip'])

In [8]:
#create a dictionary which host last capture and recent capture date
dic={}
for i in pop_raw['year'].unique():
    for j in pop_raw['year'].unique():
        if i<j:
            dic[(i,j)]=0

In [9]:
#find last capture and recent capture date for each individual
for i in pop_raw['Microchip'].unique():
    subset=pop_raw[pop_raw['Microchip']==i].copy()
    if len(subset)>=2:
        dic[tuple(subset['year'].iloc[:2].tolist())]+=1

In [10]:
#compute recaptured animals in that year
total_marked=[sum([dic[j] for j in dic if j[1]==i]) for i in pop_raw['year'].unique()]

#compute total caught animals in that year
total_caught=pop_raw.groupby('year').count()['Microchip'].tolist()

#compute the difference between caught and recaptured
total_unmarked=[total_caught[i]-total_marked[i] for i in range(len(total_caught))]

#always release animals caught
total_released=total_caught

In [11]:
total_caught_after=[]
total_recapture=[]
for i in pop_raw['year'].unique():
    year_before=[j for j in pop_raw['year'].unique() if j<i]
    year_after=[j for j in pop_raw['year'].unique() if j>i]
    
    #compute the animals caught before that year and recaptured after that year
    total_caught_after.append(sum([dic[(j,k)] for j in year_before for k in year_after]))
    
    #compute the animals caught that year and recaptured after that year
    total_recapture.append(sum([dic[(i,k)] for k in year_after]))

In [12]:
#estimate the population
pop_est=[]
for i in range(len(total_caught_after)):
    portions_animals_marked=((total_marked[i]+1)/(total_released[i]+1))
    estimated_marked_population=((total_released[i]+1)*total_caught_after[i])/(total_recapture[i]+1)+total_marked[i]
    num=estimated_marked_population/portions_animals_marked
    pop_est.append(num)

In [13]:
export=pd.DataFrame()

export['year']=pop_raw['year'].unique()

export['population']=pop_est

export.to_csv('C:/Users/tm/Downloads/utas/thesis/chapter1/channel/poems/population.csv',index=False)

export

,year,population
0,2005,0.000000
1,2011,0.000000
2,2012,7.000000
3,2013,0.000000
4,2014,0.000000
5,2018,0.000000
6,2019,108.444444
7,2020,113.050000
8,2021,120.489796
9,2022,56.318182
